<a href="https://colab.research.google.com/github/NaraaBold/DiverNaraa/blob/main/SimpleConv2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implementation of 2D Convolutional Neural Network from scratch

In [1]:
# data preperation
from keras.datasets import mnist
import numpy as np
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(X_train.shape) # (60000, 28, 28)
print(X_test.shape) # (10000, 28, 28)
print(X_train[0].dtype) # uint8

#Preprocessing 
X_train = X_train.astype(float)
X_test = X_test.astype(float)
X_train /= 255
X_test /= 255
print(X_train.max()) # 1.0
print(X_train.min()) # 0.0

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])
print(y_train.shape) # (60000,)
print(y_train_one_hot.shape) # (60000, 10)
print(y_train_one_hot.dtype) # float64

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2)
print(X_train.shape) # (48000, 28, 28)
print(X_val.shape) # (12000, 28, 28)

11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(10000, 28, 28)
uint8
1.0
0.0
(60000,)
(60000, 10)
float64
(48000, 28, 28)
(12000, 28, 28)


In [26]:
#import libraries
import math
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder

[Problem 1] Creating a 2D convolutional layer

In [19]:
class SimpleConv2d():
    def __init__(self, F, C, FH, FW, P, S,initializer=None,optimizer=None,activation=None):
        self.P = P
        self.S = S
        self.initializer = initializer
        self.optimizer = optimizer
        self.activation = activation
        self.W = self.initializer.W(F,C,FH,FW)
        self.B = self.initializer.B(F)

    def forward(self, X,debug=False):
        '''
        N - number of samples
        C - number of channel
        H - number of feature height
        W - number of feature width
        F - number of filters
        FH - filter height
        FW - filter width
        OH - output height
        OW - output width
        '''
        self.X = X
        N,C,H,W = self.X.shape
        F,C,FH,FW = self.W.shape
        OH,OW = self.output_shape2d(H,W,self.P,self.P,FH,FW,self.S,self.S)
        self.params = N,C,H,W,F,FH,FW,OH,OW
        A = np.zeros([N,F,OH,OW])
        self.X_pad = np.pad(self.X,((0,0),(0,0),(self.P,self.P),(self.P,self.P)))
        for n in range(N):
            for ch in range(F):
                for row in range(0,H,self.S):
                    for col in range(0,W,self.S):
                        if self.P == 0 and (W-2 <= col or H-2<=row):
                            continue
                        A[n,ch,row,col] = np.sum(self.X_pad[n,:,row:row+FH,col:col+FW]*self.W[ch,:,:,:]) +self.B[ch]
        if debug==True:
            return A
        else:
            return  self.activation.forward(A)
            
    def backward(self, dZ,debug=False):
        if debug==True:
            dA = dZ
        else:
            dA = self.activation.backward(dZ)
        N,C,H,W,F,FH,FW,OH,OW = self.params
        dZ = np.zeros(self.X_pad.shape)
        self.dW = np.zeros(self.W.shape)
        self.dB = np.zeros(self.B.shape)
        for n in range(N):
            for ch in range(F):
                for row in range(0,H,self.S):
                    for col in range(0,W,self.S):
                        if self.P == 0 and (W-2 <= col or H-2<=row):
                            continue
                        dZ[n,:,row:row+FH,col:col+FW] += dA[n,ch,row,col]*self.W[ch,:,:,:]
        if self.P == 0:
            dZ = np.delete(dZ,[0,H-1],axis=2)
            dZ = np.delete(dZ,[0,W-1],axis=3)
        else:
            dl_rows = range(self.P),range(H+self.P,H+2*self.P,1)
            dl_cols = range(self.P),range(W+self.P,W+2*self.P,1)
            dZ = np.delete(dZ,dl_rows,axis=2)
            dZ = np.delete(dZ,dl_cols,axis=3)
        for n in range(N):
            for ch in range(F):
                for row in range(OH):
                    for col in range(OW):
                        self.dW[ch,:,:,:] += dA[n,ch,row,col]*self.X_pad[n,:,row:row+FH,col:col+FW]
        for ch in range(F):
            self.dB[ch] = np.sum(dA[:,ch,:,:])
        self = self.optimizer.update(self)
        return dZ

    def output_shape2d(self,H,W,PH,PW,FH,FW,SH,SW):
        OH = (H +2*PH -FH)/SH +1
        OW = (W +2*PW -FW)/SW +1
        return int(OH),int(OW)

In [3]:
class SimpleInitializerConv2d:
    def __init__(self, sigma=0.01):
        self.sigma = sigma
    def W(self, F, C, FH, FW):
        return self.sigma * np.random.randn(F,C,FH,FW)
    def B(self, F):
        return np.zeros(F)

In [4]:
class ReLU:
    def forward(self, A):
        self.A = A
        return np.clip(A, 0, None)
    def backward(self, dZ):
        return dZ * np.clip(np.sign(self.A), 0, None)
class SGD:
    def __init__(self, lr):
        self.lr = lr
    def update(self, layer):
        layer.W -= self.lr * layer.dW
        layer.B -= self.lr * layer.dB
        return

[Problem 2] 2D convolutional layer experiment with small arrays

In [20]:
# CNN2 のフォワードを流す時の入力データ
# (1,1,4,4)
x = np.array([[[[ 1,  2,  3,  4],
                [ 5,  6,  7,  8],
                [ 9, 10, 11, 12],
                [13, 14, 15, 16]]]])

# (2,3,3)
w = np.array([[[[ 0.,  0.,  0.],
               [ 0.,  1.,  0.],
               [ 0., -1.,  0.]]],

              [[[ 0.,  0.,  0.],
               [ 0., -1.,  1.],
               [ 0.,  0.,  0.]]]])

simple_conv_2d = SimpleConv2d(F=2, C=1, FH=3, FW=3, P=0, S=1,initializer=SimpleInitializerConv2d(0.01),optimizer=SGD(0.01),activation=ReLU())
simple_conv_2d.W = w
A = simple_conv_2d.forward(x,True)
print(A)

[[[[-4. -4.]
   [-4. -4.]]

  [[ 1.  1.]
   [ 1.  1.]]]]


In [22]:
# (?,1,2,2,)
delta = np.array([[[[ -4,  -4],
                   [ 10,  11]],

                  [[  1,  -7],
                   [  1, -11]]]])

dZ = simple_conv_2d.backward(delta,True)
print(dZ)

[[[[-5.  4.]
   [13. 27.]]]]


[Problem 3] Output size after 2D convolution

In [41]:
class MaxPool2D():
    def __init__(self,P):
        self.P = P
        self.PA = None
        self.Pindex = None
    def forward(self,A):
        N,F,OH,OW = A.shape
        PH,PW = int(OH/self.P),int(OW/self.P)
        self.params = N,F,OH,OW,self.P,PH,PW
        self.PA = np.zeros([N,F,PH,PW])
        self.Pindex = np.zeros([N,F,PH,PW])
        for n in range(N):
            for ch in range(F):
                for row in range(PH):
                    for col in range(PW):
                        self.PA[n,ch,row,col] = np.max(A[n,ch,row*self.P:row*self.P+self.P,col*self.P:col*self.P+self.P])
                        self.Pindex[n,ch,row,col] = np.argmax(A[n,ch,row*self.P:row*self.P+self.P,col*self.P:col*self.P+self.P])
        return self.PA
        
    def backward(self,dA):
        N,F,OH,OW,PS,PH,PW = self.params
        dP = np.zeros([N,F,OH,OW])
        for n in range(N): 
            for ch in range(F):
                for row in range(PH):
                    for col in range(PW):
                        idx = self.Pindex[n,ch,row,col]
                        tmp = np.zeros((PS*PS))
                        for i in range(PS*PS):
                            if i == idx:
                                tmp[i] = dA[n,ch,row,col]
                            else:
                                tmp[i] = 0
                        dP[n,ch,row*PS:row*PS+PS,col*PS:col*PS+PS] = tmp.reshape(PS,PS)
        return dP

In [61]:
#testing
x = np.array([[[[ 1,  2,  3,  4],
                [ 5,  6,  7,  8],
                [ 9, 10, 11, 12],
                [13, 14, 15, 16]]]])
maxP = MaxPool2D(P=2)
result = maxP.forward(x)
print (result)

[[[[ 6.  8.]
   [14. 16.]]]]


[Exercise 5] (Advanced exercise) Creating an average pooling

In [24]:
class AvgPool2D():
    def __init__(self,P):
        self.P = P
        self.PA = None
        self.Pindex = None
    def forward(self,A):
        N,F,OH,OW = A.shape
        PH,PW = int(OH/self.P),int(OW/self.P)
        self.params = N,F,OH,OW,self.P,PH,PW
        self.PA = np.zeros([N,F,PH,PW])
        self.Pindex = np.zeros([N,F,PH,PW])
        for n in range(N):
            for ch in range(F):
                for row in range(PH):
                    for col in range(PW):
                        self.PA[n,ch,row,col] = np.mean(A[n,ch,row*self.P:row*self.P+self.P,col*self.P:col*self.P+self.P])
                        self.Pindex[n,ch,row,col] = np.argmax(A[n,ch,row*self.P:row*self.P+self.P,col*self.P:col*self.P+self.P])
        return self.PA
        
    def backward(self,dA):
        N,F,OH,OW,PS,PH,PW = self.params
        dP = np.zeros([N,F,OH,OW])
        for n in range(N): 
            for ch in range(F):
                for row in range(PH):
                    for col in range(PW):
                        idx = self.Pindex[n,ch,row,col]
                        tmp = np.zeros((PS*PS))
                        for i in range(PS*PS):
                            tmp[i] = dA[n,ch,row,col]/PS/PS
                        dP[n,ch,row*PS:row*PS+PS,col*PS:col*PS+PS] = tmp.reshape(PS,PS)
        return dP

In [62]:
#testing
x = np.array([[[[ 1,  2,  3,  4],
                [ 5,  6,  7,  8],
                [ 9, 10, 11, 12],
                [13, 14, 15, 16]]]])
avgP = AvgPool2D(P=2)
result = avgP.forward(x)
print (result)

[[[[ 3.5  5.5]
   [11.5 13.5]]]]


[Question 6] Smoothing

In [25]:
class Flatten:
    def __init__(self):
        pass
    def forward(self,X):
        self.shape = X.shape
        return X.reshape(len(X),-1)
    def backward(self,X):
        return X.reshape(self.shape)  

[Question 7] Learning and Estimation

In [27]:
#Defining a mini-batch generation iterator
class GetMiniBatch:
    def __init__(self, X, y, batch_size = 20, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1] 
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

In [28]:
#Defining a Weight Initialization Class
class XavierInitializer:
    def W(self, n_nodes1, n_nodes2):
        self.sigma = math.sqrt(1 / n_nodes1)
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        return W
    def B(self, n_nodes2):
        B = self.sigma * np.random.randn(n_nodes2)
        return B
    
class HeInitializer():
    def W(self, n_nodes1, n_nodes2):
        self.sigma = math.sqrt(2 / n_nodes1)
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        return W
    def B(self, n_nodes2):
        B = self.sigma * np.random.randn(n_nodes2)
        return B
        
class SimpleInitializer:
    def __init__(self, sigma):
        self.sigma = sigma
    def W(self, *shape):
        W = self.sigma * np.random.randn(*shape)
        return W
    def B(self, *shape):
        B = self.sigma * np.random.randn(*shape)
        return B

In [79]:
#Defining Gradient Update Class
class SGD:
    def __init__(self, lr):
        self.lr = lr
    def update(self, layer):
        layer.W -= self.lr * layer.dW
        layer.B -= self.lr * layer.dB
        return

class AdaGrad:
    def __init__(self, lr):
        self.lr = lr
        self.HW = 0
        self.HB = 0
    def update(self, layer):
        self.HW += layer.dW**2
        self.HB += layer.dB**2
        layer.W -= self.lr * np.sqrt(1/self.HW + 1e-7) * layer.dW
        layer.B -= self.lr * np.sqrt(1/self.HB + 1e-7) * layer.dB

In [75]:
#Defining FC layer class
class FC:
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer, Activater):
        self.optimizer = optimizer
        self.W = initializer.W(n_nodes1, n_nodes2)
        self.B = initializer.B(n_nodes2)
        self.Activer = Activater

    def forward(self, X):
        self.X = X
        A = X@self.W + self.B
        Z = self.Activer.forward(A)
        return Z
    def backward(self, dA):
        dA = self.Activer.backward(dA)
        dZ = dA@self.W.T
        self.dB = np.sum(dA, axis=0)
        self.dW = self.X.T@dA
        self = self.optimizer.update(self)
        return dZ

In [36]:
#Definition of various activation functions
class Sigmoid:
    def forward(self, A):
        self.A = A
        return self.sigmoid(A)
    def backward(self, dZ):
        _sig = self.sigmoid(self.A)
        return dZ * (1 - _sig)*_sig
    def sigmoid(self, X):
        return 1 / (1 + np.exp(-X))

class Tanh:
    def forward(self, A):
        self.A = A
        return np.tanh(A)
    def backward(self, dZ):
        return dZ * (1 - (np.tanh(self.A))**2)

class Softmax:
    def forward(self, X):
        self.Z = np.exp(X) / np.sum(np.exp(X), axis=1).reshape(-1,1)
        return self.Z
    def backward(self, Y):
        self.loss = self.loss_func(Y)
        return self.Z - Y
    def loss_func(self, Y, Z=None):
        if Z is None:
            Z = self.Z
        return (-1)*np.average(np.sum(Y*np.log(Z), axis=1))

class ReLU:
    def forward(self, A):
        self.A = A
        return np.clip(A, 0, None)
    def backward(self, dZ):
        return dZ * np.clip(np.sign(self.A), 0, None)

In [72]:
class Scratch2dCNNClassifier():
    def __init__(self, NN, CNN, n_epoch=5, n_batch=1, verbose = False):
        self.NN = NN
        self.CNN = CNN
        self.n_epoch = n_epoch
        self.n_batch = n_batch
        self.verbose = verbose
        self.log_loss = np.zeros(self.n_epoch)
        self.log_acc = np.zeros(self.n_epoch)

    def loss_function(self,y,yt):
        delta = 1e-7
        return -np.mean(yt*np.log(y+delta))

    def accuracy(self,Z,Y):
        return accuracy_score(Y,Z)

    def fit(self, X, y, X_val=False, y_val=False):
        for epoch in range(self.n_epoch):
            get_mini_batch = GetMiniBatch(X, y, batch_size=self.n_batch)
            self.loss = 0
            for mini_X_train, mini_y_train in get_mini_batch:              
                forward_data = mini_X_train[:,np.newaxis,:,:]
                for layer in range(len(self.CNN)):
                    forward_data = self.CNN[layer].forward(forward_data)
                self.flt = Flatten()
                forward_data = self.flt.forward(forward_data)
                for layer in range(len(self.NN)):
                    forward_data = self.NN[layer].forward(forward_data)
                Z = forward_data
                backward_data = (Z - mini_y_train)/self.n_batch
                for layer in range(len(self.NN)-1,-1,-1):
                    backward_data = self.NN[layer].backward(backward_data)
                backward_data = self.flt.backward(backward_data)
                for layer in range(len(self.CNN)-1,-1,-1):
                    backward_data = self.CNN[layer].backward(backward_data)
                self.loss += self.loss_function(Z,mini_y_train)
                if self.verbose:
                    print('batch loss %f'%self.loss_function(Z,mini_y_train))
            if self.verbose:
                print(self.loss/len(get_mini_batch),self.accuracy(self.predict(X),np.argmax(y,axis=1)))
            self.log_loss[epoch] = self.loss/len(get_mini_batch)
            self.log_acc[epoch] = self.accuracy(self.predict(X),np.argmax(y,axis=1))

    def predict(self, X):
        pred_data = X[:,np.newaxis,:,:]
        for layer in range(len(self.CNN)):
            pred_data = self.CNN[layer].forward(pred_data)
        pred_data = self.flt.forward(pred_data)
        for layer in range(len(self.NN)):
            pred_data = self.NN[layer].forward(pred_data)
        return np.argmax(pred_data,axis=1)

In [84]:
NN = {
    0:FC(1960, 200, HeInitializer(), SGD(0.01), Tanh()),
    1:FC(200, 200, HeInitializer(), SGD(0.01), Tanh()),
    2:FC(200, 10, SimpleInitializer(0.01), SGD(0.01), Softmax()),
}
CNN = {
    0:SimpleConv2d(F=10, C=1, FH=3, FW=3, P=1, S=1,initializer=SimpleInitializerConv2d(0.01),optimizer=SGD(0.01),activation=Tanh()),
    1:MaxPool2D(2),
}

cnn1 = Scratch2dCNNClassifier(NN=NN,CNN=CNN,n_epoch=3,n_batch=64,verbose=True)
cnn1.fit(X_train[:1000],y_train_one_hot[:1000])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if __name__ == '__main__':


batch loss 0.230074
batch loss 0.230214
batch loss 0.230352
batch loss 0.230145
batch loss 0.228094
batch loss 0.301205
batch loss 0.598638
batch loss 0.835369
batch loss 1.135721
batch loss 1.188831
batch loss 1.200005
batch loss 1.027490
batch loss 1.293009
batch loss 1.162389
batch loss 1.248332
batch loss 1.397849
0.783607310575274 0.092


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if __name__ == '__main__':


batch loss 1.336540
batch loss 1.393196
batch loss 1.322236
batch loss 1.436786
batch loss 1.585422
batch loss 1.299498
batch loss 1.380794
batch loss 1.435503
batch loss 1.418843
batch loss 1.410273
batch loss 1.295129
batch loss 1.255269
batch loss 1.183022
batch loss 1.417240
batch loss 1.293981
batch loss 1.188104
1.3532397505825289 0.1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if __name__ == '__main__':


batch loss 1.511048
batch loss 1.051109
batch loss 1.119941
batch loss 1.225520
batch loss 1.278548
batch loss 1.322582
batch loss 1.279718
batch loss 1.356407
batch loss 1.151414
batch loss 1.536098
batch loss 1.359657
batch loss 1.058692
batch loss 1.342189
batch loss 1.281108
batch loss 1.435517
batch loss 1.085387
1.2746833904480908 0.094


In [85]:
y_pred = cnn1.predict(X_val[:500])
accuracy = accuracy_score(np.argmax(y_val[:500],axis=1), y_pred)
print('accuracy:{:.3f}'.format(accuracy))

accuracy:0.090


[Question 8] (Advanced task) LeNet

In [93]:
LeNet_NN = {
    0:FC(784, 120, HeInitializer(), SGD(0.01), Sigmoid()),
    1:FC(120, 84, HeInitializer(), SGD(0.01), Sigmoid()),
    2:FC(84, 10, SimpleInitializer(0.01), SGD(0.01), Softmax()),
}
LeNet_CNN = {
    0:SimpleConv2d(F=6, C=1, FH=5, FW=5, P=2, S=1,initializer=SimpleInitializerConv2d(0.01),optimizer=SGD(0.01),activation=Sigmoid()),
    1:MaxPool2D(2),
    2:SimpleConv2d(F=16, C=6, FH=5, FW=5, P=2, S=1,initializer=SimpleInitializerConv2d(0.01),optimizer=SGD(0.01),activation=Sigmoid()),
    3:MaxPool2D(2),
}

cnn2 = Scratch2dCNNClassifier(NN=LeNet_NN,CNN=LeNet_CNN,n_epoch=3,n_batch=64,verbose=True)
cnn2.fit(X_train[:1000],y_train_one_hot[:1000])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if __name__ == '__main__':


batch loss 0.230808
batch loss 0.230165
batch loss 0.230313
batch loss 0.230459
batch loss 0.229922
batch loss 0.230795
batch loss 0.238916
batch loss 0.250098
batch loss 0.410887
batch loss 0.461765
batch loss 0.460870
batch loss 0.430543
batch loss 0.500187
batch loss 0.598813
batch loss 0.664161
batch loss 0.786755
0.3865909414448499 0.093


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if __name__ == '__main__':


batch loss 0.671285
batch loss 0.723003
batch loss 0.817410
batch loss 0.702172
batch loss 0.732150
batch loss 0.842478
batch loss 0.679857
batch loss 0.839863
batch loss 0.779890
batch loss 0.722245
batch loss 0.672916
batch loss 0.879119
batch loss 0.872802
batch loss 1.022017
batch loss 0.919089
batch loss 1.047281
0.8077234725358485 0.097


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if __name__ == '__main__':


batch loss 0.929558
batch loss 0.946338
batch loss 1.055599
batch loss 1.022768
batch loss 0.947137
batch loss 1.072598
batch loss 0.842164
batch loss 0.998882
batch loss 0.974568
batch loss 1.091302
batch loss 0.704805
batch loss 0.928820
batch loss 0.973565
batch loss 0.807721
batch loss 0.799134
batch loss 0.600643
0.9184751065479813 0.099


In [94]:
y_pred = cnn2.predict(X_val[:500])
accuracy = accuracy_score(np.argmax(y_val[:500],axis=1), y_pred)
print('accuracy LeNet:{:.3f}'.format(accuracy))

accuracy LeNet:0.076


[Problem 10] Calculation of output size and number of parameters

In [103]:
#Output size = (input size + 2 x padding size - filter size) / stride size + 1 
#number of parameters = filter size x input channel x output channel + output channel

def calculate_size_paramaters (input_H, input_W, padding, filter_H, filter_W, stride, input_channel, output_channel):
  #output_size = (input_H*input_W + 2 * padding - filter_H*filter_W) / (stride + 1)
  output_height = (input_H + 2 * padding  - filter_H) / stride + 1
  output_width = (input_W + 2 * padding  - filter_W) / stride + 1
  parameters = filter_H * filter_W * input_channel * output_channel + output_channel
  #print (output_height)
  #print (output_width)
  return output_height*output_width*output_channel, parameters

in_H = 144 # input height
in_W = 144 # input width
pad = 0 # padding
stride = 1 # stride
in_channel = 3 #input channels
filter_H = 3 # filter height
filter_W = 3 # filter width
out_channel = 6 # out channels
print ("Case 1: " + str(calculate_size_paramaters(in_H, in_W, pad, filter_H, filter_W, stride, in_channel, out_channel)))

Case 1: (120984.0, 168)


In [104]:
in_H = 60 # input height
in_W = 60 # input width
pad = 0 # padding
stride = 1 # stride
in_channel = 24 #input channels
filter_H = 3 # filter height
filter_W = 3 # filter width
out_channel = 48 # out channels
print ("Case 2: " + str(calculate_size_paramaters(in_H, in_W, pad, filter_H, filter_W, stride, in_channel, out_channel)))

Case 2: (161472.0, 10416)


In [105]:
in_H = 20 # input height
in_W = 20 # input width
pad = 0 # padding
stride = 2 # stride
in_channel = 10 #input channels
filter_H = 3 # filter height
filter_W = 3 # filter width
out_channel = 20 # out channels
print ("Case 3: " + str(calculate_size_paramaters(in_H, in_W, pad, filter_H, filter_W, stride, in_channel, out_channel)))

Case 3: (1805.0, 1820)
